Word Document Formatter

1. importing libraries

In [1]:
from langgraph.graph import StateGraph
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from typing import TypedDict, Annotated, List, Dict, Optional, Literal
from langgraph.graph import add_messages
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

In [2]:
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
model = ChatOpenAI(
    model="mistralai/devstral-2512:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key="",
)

2.states define

In [4]:
class DocStructure(BaseModel):
    document_type: str = Field(
        description="Type of document: academic, research, business, general"
    )

    headings: Dict[str, int] = Field(
        description="Detected headings mapped to heading level (e.g., {'Introduction': 1})"
    )

    body_font_size: int = Field(
        description="Target font size for body text"
    )

    heading_font_size: int = Field(
        description="Target font size for headings"
    )

    alignment: str = Field(
        description="Text alignment: left, right, center, justify"
    )

    issues_found: List[str] = Field(
        description="Formatting or structural issues detected"
    )


In [5]:
class FormatCommand(BaseModel):
    action: str
    target: str
    value: int | str

In [6]:
class wordFormatter(TypedDict, total=False):
    doc: str
    structDoc: Annotated[list, lambda x, y: y]
    formatPlan: List[DocStructure]
    formatCmd: List[FormatCommand]
    userCmd: List[FormatCommand]
    output_doc: Optional[str]
    output_pdf: Optional[str]
    user_instruction: str

3. nodes Building

NodeFn1: Loading the document

In [7]:
from docx import Document

def load_docx(path: str):
    doc = Document(path)
    paragraphs = []

    for p in doc.paragraphs:
        paragraphs.append({
            "text": p.text.strip(),
            "style": p.style.name if p.style else None,
            "alignment": p.alignment,
            "runs": [
                {
                    "text": r.text,
                    "bold": r.bold,
                    "italic": r.italic,
                    "font_size": r.font.size.pt if r.font.size else None
                }
                for r in p.runs
            ]
        })
    return paragraphs


In [8]:
loader = load_docx('Draft.docx')
# doc = loader.load()
print(loader)

[{'text': 'Introduction', 'style': 'Heading 1', 'alignment': None, 'runs': [{'text': 'Introduction', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'Qwertyujnbvcddx', 'style': 'List Paragraph', 'alignment': None, 'runs': [{'text': 'Qwertyujnbvcddx', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'Ertyhhbvcx', 'style': 'List Paragraph', 'alignment': None, 'runs': [{'text': 'Ertyhhbvcx', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'sdfgbhcd', 'style': 'List Paragraph', 'alignment': None, 'runs': [{'text': 'sdfgbhcd', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'This document is created for a word processing formatting project. The purpose of this file is to allow the user to apply formatting such as text justification, font size, font color, line spacing, and paragraph alignment.', 'style': 'Normal', 'alignment': None, 'runs': [{'text': 'This ', 'bold': None, 'italic': None, 'font_size': None}, {'text': 'document is created fo

In [9]:
def loadDocNode(state: wordFormatter) -> wordFormatter:
    doc = state['doc']
    state['structDoc'] = load_docx(doc)
    return state

NodeFn2: Structure Analysis Agent node

In [10]:
def docStructureNode(state: wordFormatter) -> wordFormatter:
    struct_doc = state["structDoc"]

    parser = PydanticOutputParser(pydantic_object=DocStructure)

    prompt = PromptTemplate(
        template="""
You are a document structure analysis agent.

Analyze the following document structure and produce a formatting plan.

Document structure:
{struct_doc}

{format_instructions}
""",
        input_variables=["struct_doc"],
        partial_variables={
            "format_instructions": parser.get_format_instructions()
        }
    )

    response = model.invoke(
        prompt.format(struct_doc=struct_doc)
    )

    state["formatPlan"] = parser.parse(response.content)
    return state

NodeFn3: Formatter Agent node

3.1 word formatter

In [11]:
# from docx import Document
# from docx.enum.text import WD_ALIGN_PARAGRAPH
# from docx.shared import Pt

# def formatterNode(state: wordFormatter) -> wordFormatter:
#     plan = state["formatPlan"]
#     doc_path = state["doc"]

#     doc = Document(doc_path)

#     for p in doc.paragraphs:
#         # Headings
#         if p.style.name.startswith("Heading"):
#             for run in p.runs:
#                 run.font.size = Pt(plan.heading_font_size)
#         else:
#             # Body text
#             p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
#             for run in p.runs:
#                 run.font.size = Pt(plan.body_font_size)

#     output_path = "formatted.docx"
#     doc.save(output_path)

#     state["output_doc"] = output_path
#     return state

3.2 pdf generater

In [12]:
import subprocess

def pdfGeneratorNode(state: wordFormatter):
    docx_path = state["output_doc"]
    pdf_path = docx_path.replace(".docx", ".pdf")

    subprocess.run([
        "libreoffice",
        "--headless",
        "--convert-to", "pdf",
        docx_path,
        "--outdir", "."
    ], check=True)

    return {"output_pdf": pdf_path}


NodeFn4: User Instructions node

In [13]:
def chatCommandNode(state: wordFormatter):
    instruction = state["user_instruction"]

    prompt = PromptTemplate(
        template="""
        Convert the user instruction into a formatting command.
        Instruction: {instruction}
        """,
        input_variables=["instruction"]
    )
    formatted_prompt = prompt.format(instruction=instruction)
    command = model.with_structured_output(FormatCommand).invoke(formatted_prompt)
    
    state["UserCmd"] = command
    return state

NodeFn5: Chat Command Formatter Node

In [14]:
# from docx.enum.text import WD_ALIGN_PARAGRAPH

# from docx import Document
# from docx.shared import Pt
# from docx.enum.text import WD_ALIGN_PARAGRAPH

# def applyChatFormatNode(state: wordFormatter) -> wordFormatter:
#     doc = Document(state["doc"])
#     instruction = state.get("user_instruction", "").lower()

#     for p in doc.paragraphs:
#         if "justify" in instruction:
#             p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

#         if "font size 12" in instruction:
#             for run in p.runs:
#                 run.font.size = Pt(12)

#     output_path = "formatted.docx"
#     doc.save(output_path)

#     state["output_doc"] = output_path
#     return state



NodeFn7: ToolcndNode

In [15]:
def ToolcmdNode(state: wordFormatter) -> wordFormatter:

    userCmd = state['userCmd']
    formatPlan = state['formatPlan']
    prompt = PromptTemplate(
        template="""
        Convert the instruction into a formatting command. give preference to user command and then formatPlan
        Instruction: User command: {userCmd}, FormatPlan : {formatPlan}
        """,
        input_variables=["userCmd", "formatPlan"]
    )
    formatted_prompt = prompt.format(userCmd = userCmd, formatPlan = formatPlan)
    command = model.with_structured_output(FormatCommand).invoke(formatted_prompt)
    state['formatCmd'] = command

    return state

Conditinal path define

In [16]:
def route_after_load(state: wordFormatter):
    if state.get("user_instruction"):
        return "chat_command"
    return "doc_structure"

4.Graph Struct

In [17]:
graph = StateGraph(wordFormatter)

graph.add_node("load_docx", loadDocNode)
graph.add_node("doc_structure", docStructureNode)
graph.add_node("doc_formatter", formatterNode)
graph.add_node("doc_pdf", pdfGeneratorNode)
graph.add_node("chat_command", chatCommandNode)
graph.add_node("chat_format", applyChatFormatNode)
graph.add_node('Toolcmd', ToolcmdNode)

graph.set_entry_point("load_docx")
# graph.add_conditional_edges("load_docx",route_after_load,{
#         "chat_command": "chat_command",
#         "doc_structure": "doc_structure"
#     }
# )

graph.add_edge("load_docx", "doc_structure")
graph.add_edge("load_docx", "chat_command")

graph.add_edge("chat_command", "Toolcmd")
graph.add_edge("doc_structure", "Toolcmd")


# graph.add_edge("doc_formatter", "doc_pdf")
# graph.add_edge("chat_format", "doc_pdf")
graph.set_finish_point("Toolcmd")

app = graph.compile()


NameError: name 'formatterNode' is not defined

In [ ]:
app.get_graph().print_ascii()

In [ ]:
# result = app.invoke({
#      "doc": "Draft.docx",
#      "structDoc": [],
#      "formatPlan": [],
#      "output_doc": []
#  })

# print(result["output_doc"])
# print(result["structDoc"])
# print(result["formatPlan"])


In [ ]:
res = app.invoke({
    "doc": "Draft.docx",
    "user_instruction": "Make font size 20"
})

In [ ]:
print(res['formatCmd'])